# Applications of Convex.jl in Optimization Involving Complex Numbers

## Ayush Pandey | JuliaCon 2017

[https://github.com/Ayush-iitkgp/JuliaCon2017Presentation](https://github.com/Ayush-iitkgp/JuliaCon2017Presentation)

<!--- http://www.damian.oquanta.info/posts/make-your-slides-with-ipython.html --->

# About Me 

* **BS and MS in Mathematics and Computing Sciences (July'12-June'17) at Indian Institute of Technology Kharagpur **

* **Google Summer of Code 2016 and 2017 student under the Julia Language**

* **GitHub:** [Ayush-iitkgp](https://github.com/ayush-iitkgp)

* **Website:** [https://ayush-iitkgp.github.io](https://ayush-iitkgp.github.io/) 



<!----#### Blogging my GSoC'16 experience at [http://ayush-iitkgp.rhcloud.com](http://ayush-iitkgp.rhcloud.com/) ---->

<!--- ## CVX.jl team

* [CVX.jl](https://github.com/cvxgrp/CVX.jl): Madeleine Udell, Karanveer Mohan, David Zeng, Jenny Hong
<!---* [ParallelSparseMatMul.jl](https://github.com/madeleineudell/ParallelSparseMatMul.jl): Madeleine Udell
--->

# Outline

* Power Flow Optimization
* Fidelity in Quantum Information
* Benchmark

### Optimal Power Flow - Introduction

* Power flow study is an analysis of a connected electrical power system’s capability to adequately supply the connected load.  

![An example of a power network](Power_Network.png)

* **Unknowns: ** - Voltages angle and magnitude information for each bus
* **Knowns: ** - Load( such as appliances and lights.) and generator real power and voltage condition.
<!-- An electrical load is an electrical component or portion of a circuit that consumes (active) electric power. -->

### Optimal Power Flow - Mathematical Formulation

#### Constraints

![Each transmission line has four flows](Power_Network_2.png)

* $p_{ij}$: Active power entering the line from node i
* Let $x_{i}$ denote the complex voltage for node i of the network. 

We have the following power balance equations which are **non-linear** in unknown x.

![Power balance equations](Power_Network_3.png)

#### Objective 

Depends on the business needs such as: 

* Minimize power losses in an electrical network
* Minimize cost of generation

### Optimal Power Flow - SDP Relaxation

* The original optimal power flow problem is non-convex in nature.
* Thanks to the **lifting technique** which converts the above optimization problem to a SemiDefinite Programming Problem.
* The relaxed SDP problem finds the near global solution of the original non-convex problem.

### Example

* The data is taken from the IEEE 14 Bus test case which represents a portion of the American Electric Power System (in the Midwestern US) as of February, 1962.

In [ ]:
using Convex  # Read the input data
using FactCheck
using MAT   #Pkg.add("MAT")
TOL = 1e-2;
input = matopen("Data.mat")
varnames = names(input)
Data = read(input, "inj","Y");
n=size(Data[2],1); # Create some intermediate variables
Y=Data[2];
inj=Data[1];

In [4]:
W = ComplexVariable(n,n); # W is the matrix of pairwise products of the voltages

objective = real(sum(diag(W))); # The objective is to minimize cost of generation

c1 = Constraint[]; # The constraints are power balance equations
for i=2:n
    push!(c1,sum(W[i,:].*(Y[i,:]'))==inj[i]);
end
c2 = W in :SDP
c3 = real(W[1,1])==1.06^2;
push!(c1, c2)
push!(c1, c3)

Variable of
size: (4, 2)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()

In [ ]:
p = maximize(objective,c1); # Create the problem
solve!(p) # Solve the problem
p.optval #15.125857662600703
evaluate(objective) #15.1258578588357

In [5]:
output = matopen("Res.mat") # Verify the results
names(output)
outputData = read(output, "Wres");
Wres = outputData
real_diff = real(W.value) - real(Wres);
imag_diff = imag(W.value) - imag(Wres);
@fact real_diff => roughly(zeros(n,n), TOL)
@fact imag_diff => roughly(zeros(n,n), TOL)

AbstractExpr with
head: geomean
size: (1, 1)
sign: Convex.Positive()
vexity: Convex.ConcaveVexity()


### Fidelity in Quantum Information Theory - Introduction

* This example is inspired from a lecture of John Watrous in the [course on Theory of Quantum Information](https://cs.uwaterloo.ca/~watrous/CS766/LectureNotes/08.pdf).

* Fidelity is a measure of the **closeness** of two quantum states.
   
* The ability to distinguish between the quantum states is equivalent to the ability to distinguish between the classical probability distributions. 

* If fidelity between two states is 1, they are the same quantum state.

<!-- If an experimenter is attempting to determine whether a quantum state is either of two possibilities {\displaystyle \rho } \rho  or {\displaystyle \sigma } \sigma , the most general possible measurement they can make on the state is a POVM, which is described by a set of Hermitian positive semidefinite operators {\displaystyle \{F_{i}\}} \{F_{i}\}. If the state given to the experimenter is {\displaystyle \rho } \rho , they will witness outcome {\displaystyle i} i with probability {\displaystyle p_{i}=\mathrm {Tr} [\rho F_{i}]} p_{i}={\mathrm  {Tr}}[\rho F_{i}], and likewise with probability {\displaystyle q_{i}=\mathrm {Tr} [\sigma F_{i}]} q_{i}={\mathrm  {Tr}}[\sigma F_{i}] for {\displaystyle \sigma } \sigma  -->

<!-- Wikipedia Link -->
<!-- https://en.wikipedia.org/wiki/Fidelity_of_quantum_states -->

* **Application** 

![Quantum Cryptography](Fidelity.png)

* The Fidelity between two Hermitian semidefinite matrices P and Q is defined as:

$$F(P,Q) = {||{P}^{1/2}{Q}^{1/2} ||}_{tr} =  \max\; |trace({P}^{1/2}U{Q}^{1/2})|$$

where the trace norm $||.||_{tr}$ is the sum of the singular values, and the maximization goes over the set of all unitary matrices U.

### Fidelity in Quantum Information Theory - Mathematical Formulation

Fidelity can be expressed as the optimal value of the following complex-valued SDP:

$$ \textbf{maximize} \frac{1}{2} trace(Z+Z^*)$$

$$\text{subject to } \left[\begin{array}{cc}P&Z\\{Z}^{*}&Q\end{array}\right] \succeq 0$$

$$\text{where } Z \in \mathbf {C}^{n \times n}$$

### Example

In [ ]:
n = 20 # Create the data
P = randn(n,n) + im*randn(n,n)
P = P*P'
Q = randn(n,n) + im*randn(n,n)
Q = Q*Q'

Z = ComplexVariable(n,n) # Declare convex variable

objective = 0.5*real(trace(Z+Z'))  # Specify the problem
constraint = [P Z;Z' Q] ⪰ 0
problem = maximize(objective,constraint)
 
solve!(problem) # Solve the problem
computed_fidelity = evaluate(objective)

In [9]:
# Verify that computer fidelity is equal to actual fidelity
P1,P2 = eig(P)
sqP = P2 * diagm([p1^0.5 for p1 in P1]) * P2'
Q1,Q2 = eig(Q)
sqQ = Q2 * diagm([q1^0.5 for q1 in Q1]) * Q2'
objective = 2*x + 1 - sqrt(sum(y))
p = minimize(objective, constraint1, constraint2)
actual_fidelity = sum(svd(sqP * sqQ)[2])

Problem:
minimize AbstractExpr with
head: +
size: (1, 1)
sign: Convex.NoSign()
vexity: Convex.ConvexVexity()

subject to
Constraint:
== constraint
lhs: AbstractExpr with
head: *
size: (3, 1)
sign: Convex.NoSign()
vexity: Convex.AffineVexity()

rhs: [-1.6048077944321295,2.1044876445367,0.6535176954608789]
vexity: Convex.AffineVexity()
		Constraint:
<= constraint
lhs: AbstractExpr with
head: norm2
size: (1, 1)
sign: Convex.Positive()
vexity: Convex.ConvexVexity()

rhs: 2
vexity: Convex.ConvexVexity()
current status: not yet solved

# Thank You!